In [ ]:

import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover

In [2]:
data_csv = pd.read_csv('USDJPY60(2008 - 2024).csv', delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0)
data_csv.index = pd.to_datetime(data_csv.index)


In [4]:
data_csv

,Open,High,Low,Close,Volume
2008-01-04 15:00:00,108.280,108.495,108.145,108.220,67464
2008-01-04 16:00:00,108.220,108.490,108.200,108.380,93299
2008-01-04 17:00:00,108.380,108.855,108.375,108.560,62697
2008-01-04 18:00:00,108.555,108.575,108.370,108.520,75523
2008-01-04 19:00:00,108.525,108.525,108.350,108.465,62660
...,...,...,...,...,...
2024-01-30 01:00:00,147.298,147.430,147.247,147.313,14953
2024-01-30 02:00:00,147.317,147.443,147.299,147.438,11702
2024-01-30 03:00:00,147.438,147.483,147.369,147.393,11595
2024-01-30 04:00:00,147.392,147.401,147.306,147.361,12097


In [ ]:
ticker = "USDJPY=X"
data = yf.download(ticker, period="1y", interval="1h")


In [ ]:
class EmaCrossover(Strategy):
    upperbound = 70
    upperbound2 = 60
    lowerbound = 30
    lowerbound2 = 40
    
    def init(self):
    
        self.rsi = self.I(ta.rsi, self.data.df['Close'], 14)

    def next(self):
        if crossover(self.rsi, self.upperbound):            
            self.buy()
        elif crossover(self.rsi, self.upperbound2):
            self.position.close()
        elif crossover(self.rsi, self.lowerbound):
            self.sell()
        elif crossover(self.rsi, self.lowerbound2):
            self.position.close()


In [ ]:
bt = Backtest(data, EmaCrossover, cash=10000, commission=.002)
output = bt.run()
print(output)
bt.plot()

In [ ]:
data_csv['emafast'] = ta.ema(close = data_csv['Close'], length=9)
data_csv['emaslow'] = ta.ema(close = data_csv['Close'], length=15)

In [ ]:
help(ta.rsi)